## Format data

In [196]:
from os import walk

filenames = next(walk('./data/input'), (None, None, []))[2]  # [] if no file

print(filenames)

['emAvecVous.txt', 'aec2020.txt', 'zemmour2022.txt']


In [197]:
raw_contents = {}

for filename in filenames:
    with open(f"./data/input/{filename}", 'r') as file:
        raw_contents[filename] = file.read()

with open('./data/stop-words/fr.txt') as file:
    stop_words = file.read().splitlines()

# print([content[0:100] for filename, content in raw_contents.items()])


In [198]:
import re

file_words = {}

for filename, raw_content in raw_contents.items():
    words = re.sub("[!,’:%«»•())“/\-\.\s\d]+", "\n", raw_content).lower().split()
    file_words[filename] = words

# print([words[0:10] for filename, words in file_words.items()])

In [199]:
file_filtered_words = {}

for filename, words in file_words.items():
    filtered_words = [word for word in words if word not in stop_words ]
    file_filtered_words[filename] = filtered_words

# print([filtered_words[0:10] for filename, filtered_words in file_filtered_words.items()])



In [200]:
file_count_dicts = {}

for filename, filtered_words in file_filtered_words.items():
    count_dict = {}
    for current_word in filtered_words:
        if current_word not in count_dict:
            count = len([word for word in filtered_words if word == current_word ])
            count_dict[current_word] = count
    
    file_count_dicts[filename] = count_dict

# print([list(count_dict.keys())[0:10] for filename, count_dict in file_count_dicts.items()])


In [201]:
results = {}

for filename, count_dict in file_count_dicts.items():
    count_list = list(count_dict.items())
    count_list.sort(key=lambda x: x[1], reverse=True)
    result = dict(count_list)
    results[filename] = result

# print([list(result.keys())[0:10] for filename, result in results.items()])


In [202]:
import json

for filename, result in results.items():
    output_filename = filename.replace('.txt', '.json')

    with open(f'./data/output/{output_filename}', 'w', encoding='utf-8') as file:
        json.dump(result, file, ensure_ascii=False, indent=2)